# Importing libraries

In [ ]:
import os, glob, json, csv, subprocess, sys, re
from urllib.request import urlopen
from itertools import chain
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Counter
import pandas as pd
import plotly.graph_objs as go
import plotly
from plotly.offline import plot, init_notebook_mode, iplot
import itertools

%matplotlib inline

# Defining directories

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/openjpa/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_openjpa/'

# Defining git command function

In [ ]:
def execute_shell_command(buglink, id1, id2, cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return buglink, id1, id2, out, error
    pipe.wait()

# Load data from CSV file

In [ ]:
dfhist = pd.read_csv(analyze_dir + '03_file-diff/diffhist-bugs.csv')
dfmin = pd.read_csv(analyze_dir + '03_file-diff/diffmin-bugs.csv')
dfmyers = pd.read_csv(analyze_dir + '03_file-diff/diffmyers-bugs.csv')
dfpat = pd.read_csv(analyze_dir + '03_file-diff/diffpat-bugs.csv')

Defining columns

In [ ]:
cols = ['bug_id','bugintro_commitID','parent_id','#insertions','#deletions','filename','#line_changed']
column = ['bug_id','bugintro_commitID','parent_id','filename','#deletions']

# Removing files without buglines

In [ ]:
dthist = dfhist[cols][dfhist[cols]['#deletions'] != 0]
dtmin = dfmin[cols][dfmin[cols]['#deletions'] != 0]
dtmyers = dfmyers[cols][dfmyers[cols]['#deletions'] != 0]
dtpat = dfpat[cols][dfpat[cols]['#deletions'] != 0]
dtpat

In [ ]:
print ("Number of data <histogram>: %i" %(len(dthist)))
print ("Number of data <minimal>: %i" %(len(dtmin)))
print ("Number of data <myers>: %i" %(len(dtmyers)))
print ("Number of data <patience>: %i" %(len(dtpat)))

List of filename with deleted lines is saved into CSV file

In [ ]:
dt_algorithm = [dthist, dtmin, dtmyers, dtpat]
algorithms = ['histogram','minimal','myers','patience']

In [ ]:
for n, dt in enumerate(dt_algorithm):
    d_alg = []
    for c in range(0, len(dt)):
        sys.stdout.write("\rWriting data for algorithm {}, no: {} / {}".format((n+1), (c+1), len(dt)))
        sys.stdout.flush()
        item = dt.iloc[c]
        tem = [item[0],item[1],item[2],item[5],item[4]]
        d_alg.append(tem)
        
    #save into CSV file:
    with open(analyze_dir + '03_file-diff/01_filename/' + algorithms[n] + '_files.csv','w') as csvfile:
        writers = csv.writer(csvfile, delimiter=",")
        writers.writerow(column)
        for zz in d_alg:
            writers.writerow(zz)
    print(" ==> File '" + algorithms[n] + "_files.csv' has been created")

# Summarize total buglines for each algorithm

In [ ]:
fname = pd.read_csv(analyze_dir + "03_file-diff/01_filename/histogram_files.csv")
fname = fname[column]
summ = fname['#deletions'].sum()
summ

In [ ]:
total_bugline = []
n = 0
for file in glob.iglob(analyze_dir + "03_file-diff/01_filename/*", recursive=True):
    fname = pd.read_csv(file)
    fname = fname[column]
    sum_bugs = fname['#deletions'].sum()
    file = re.search("\/(\w+)\_files", file)
    file = file.groups()
    temp = [file[0], sum_bugs]
    total_bugline.append(temp)
    n += 1
total_bugline.sort()
total_bugline

In [ ]:
with open(analyze_dir + "03_file-diff/01_filename/total_bugline.csv","w") as myfile:
    cols = ['diff_algorithm','total_bugline']
    writers = csv.writer(myfile, delimiter=",")
    writers.writerow(cols)
    for ss in total_bugline:
        writers.writerow(ss)
print ("File 'total_bugline' has been created")

In [ ]:
dftotal = pd.read_csv(analyze_dir + "03_file-diff/01_filename/total_bugline.csv")
dftotal = dftotal[cols]
dftotal

In [ ]:
graphtotal = go.Bar(
    x = dftotal['diff_algorithm'],
    y = dftotal['total_bugline'],
    name = dftotal['diff_algorithm'],
    text = dftotal['total_bugline'],
    textposition = 'auto',
    marker = dict(
        color = 'rgb(150,255,200)',
        line = dict(
            color='rgb(8,48,107)',
            width=1.5),
    ),
    opacity=0.6
)

dt = [graphtotal]
layout = go.Layout(
    title = "Total number of bugline for each algorithm in OPENJPA Project"
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)